# Data Enrichment Process
In this notebook, we are working on enriching the Goodreads10k dataset. We are describing every step in detail.

## 0. Import Libraries
First, we are importing libraries that are needed for our process.

In [1]:
import time
from qwikidata.sparql  import return_sparql_query_results
from qwikidata.linked_data_interface import get_entity_dict_from_api
import pandas as pd
import numpy as np
import pickle as pkl
from collections import Counter
import urllib.request
import json
import string
from difflib import SequenceMatcher
from viapy.api import ViafAPI
import re
import math
import matplotlib.pyplot as plt
from scipy.signal import find_peaks

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

We are setting the locations where we are saving our output files. The "large" location is used for files too large to be added on github - hence they are ignored during the git commit.

In [2]:
location_to_save = "data/goodbooks-10k/final_files/"
large_location_to_save = "data/goodbooks-10k/final_files/large_files/"

## 1. Download all WikiData entries
<b>Goal</b>: To download (using SparQL queries) <u>all WikiData entries that have VIAF ids</u>. Presumably, these are all the authors contained in WikiData, and their VIAF id will be used later on to link them to our dataset. 

A. Get all WikiData entries with VIAF ID.

In [3]:
query_string = """
        SELECT DISTINCT ?item ?viaf  # return QID and VIAF ID 
WHERE 
{ ?item wdt:P214 ?viaf. # select all the items in WikiData that have VIAF ID
}
        """
start = time.time()
items_with_viaf = return_sparql_query_results(query_string) # run the query and get results
print("It took "+str(np.round(time.time()-start,2))+ " seconds.")

It took 65.94 seconds.


B. Process results. We end up only keeping their QID (WikiData identification) and VIAF id.

In [4]:
items_with_viaf = pd.DataFrame(items_with_viaf["results"]["bindings"])
items_with_viaf["item"] = items_with_viaf["item"].apply(lambda x: x["value"].split("/")[-1]) # keep only QID
items_with_viaf["viaf"] = items_with_viaf["viaf"].apply(lambda x: x["value"]) # keep only VIAF ID
items_with_viaf.columns = ["QID", "viaf_id"] # rename columns

In [5]:
items_with_viaf.head()

,QID,viaf_id
0,Q289693,135150789
1,Q481146,14775085
2,Q507746,6650
3,Q568833,158264550
4,Q573519,2274


In [6]:
print("In WikiData, there are "+str(len(items_with_viaf))+" items with VIAF ID.")

In WikiData, there are 3033762 items with VIAF ID.


C. Save results.

In [7]:
# save with Pickle
with open(large_location_to_save + "items_with_viaf_wikidata.pkl", "wb") as output_file:
    pkl.dump(items_with_viaf, output_file)

## 3. First look in the data
<b>Goal</b>: To do a first examination on how the data looks like.

A. Read book file.

In [3]:
filename = "data/goodbooks-10k/books.csv" # Books-Crossing books
books = pd.read_csv(filename, low_memory = False) # read books
#books.columns = ["ISBN", "title", "author", "year", "publisher"] # rename columns to simplify

In [4]:
print("There are "+str(len(books))+" books.")

There are 10000 books.


In [5]:
books.head()

,book_id,goodreads_book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,ratings_count,work_ratings_count,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url
0,1,2767052,2767052,2792775,272,439023483,9.780439e+12,Suzanne Collins,2008.0,The Hunger Games,...,4780653,4942365,155254,66715,127936,560092,1481305,2706317,https://images.gr-assets.com/books/1447303603m...,https://images.gr-assets.com/books/1447303603s...
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,4602479,4800065,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...
2,3,41865,41865,3212258,226,316015849,9.780316e+12,Stephenie Meyer,2005.0,Twilight,...,3866839,3916824,95009,456191,436802,793319,875073,1355439,https://images.gr-assets.com/books/1361039443m...,https://images.gr-assets.com/books/1361039443s...
3,4,2657,2657,3275794,487,61120081,9.780061e+12,Harper Lee,1960.0,To Kill a Mockingbird,...,3198671,3340896,72586,60427,117415,446835,1001952,1714267,https://images.gr-assets.com/books/1361975680m...,https://images.gr-assets.com/books/1361975680s...
4,5,4671,4671,245494,1356,743273567,9.780743e+12,F. Scott Fitzgerald,1925.0,The Great Gatsby,...,2683664,2773745,51992,86236,197621,606158,936012,947718,https://images.gr-assets.com/books/1490528560m...,https://images.gr-assets.com/books/1490528560s...


In [7]:
books = books.rename({"authors":"author"}, axis=1)

B. Choose "unique" authors based on exact author name.

In [8]:
authors = pd.DataFrame(books.author.unique(), columns=["author"]).dropna().reset_index().drop("index", axis=1) # find unique authors and remove nan values

In [9]:
print("There are "+str(len(authors))+" unique authors.")

There are 4664 unique authors.


In [10]:
authors.head()

,author
0,Suzanne Collins
1,"J.K. Rowling, Mary GrandPré"
2,Stephenie Meyer
3,Harper Lee
4,F. Scott Fitzgerald


C. Statistics.

C.1 Non-latin named authors. We are looking into how many authors have legible names in English.

In [11]:
def isEnglish(s): # function to check if a name is latin
    try:
        s.encode(encoding='utf-8').decode('ascii')
    except UnicodeDecodeError:
        return False
    else:
        return True

In [12]:
authors["author_latin"] = authors.author.apply(lambda x : isEnglish(x))
num_latin_authors = len(authors[authors.author_latin == True])

In [13]:
print("There are ", num_latin_authors,"latin authors out of", str(len(authors))+".")

There are  4417 latin authors out of 4664.


In [14]:
del authors["author_latin"]

C.2 Author entries with a single name. We are looking into how many authors are only referenced by one name (e.g., their last name).

In [15]:
authors["author_single"] = authors.author.apply(lambda x: len(x.split(" "))==1)
num_single_named_authors = len(authors[authors.author_single == True])

In [16]:
authors[authors.author_single == True]

,author,author_single
325,Anonymous,True
365,Banksy,True
395,Voltaire,True
973,Avi,True
1118,Amazon,True
1408,Sapphire,True
2075,Aristotle,True
2187,Tijan,True
2433,BookRags,True
2771,Zane,True


In [17]:
print("There are", num_single_named_authors,"unique single named authors out of", str(len(authors))+".")

There are 15 unique single named authors out of 4664.


In [18]:
del(authors["author_single"])

## 4. Access Google Books API
<b>Goal</b>: To use the ISBNs from the dataset to get <u>the author and title of the book from <b>Google Books</b></u>. The reason is that the Book-Crossing dataset seems relatively incosistent in terms of naming.

A. Function to access the GB API.

In [19]:
def get_author_title(ISBN):
    base_api_link = "https://www.googleapis.com/books/v1/volumes?q=isbn:"

        
    with urllib.request.urlopen(base_api_link + ISBN) as f:
        text = f.read()
    decoded_text = text.decode("utf-8")
    obj = json.loads(decoded_text) # deserializes decoded_text to a Python object
    try:
        volume_info = obj["items"][0] 
    except: 
        
        return("","")
    title = volume_info["volumeInfo"]["title"]
    try:
        authors = volume_info["volumeInfo"]["authors"]
        
    except:
        return("",title)

    return(authors,title)

B. Initialize the entries.

In [24]:
books["alt_title"] = ""
books["alt_author"] = ""

C. Run the requests.

In [ ]:
start = time.time()
for index,row in books.iterrows():
    if row["alt_author"]=="":
        ISBN = row["isbn"]
        print(row.original_title)
        statement = True
        i=0
        while statement:

            try:
                authors, title = get_author_title(ISBN)
                statement = False
            except:
                if i>50:
                    statement = False
                    print("IT FAILED MORE THAN 50 TIMES!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
                time.sleep(0.1)
            i+=1  

        if type(authors)==list:
            authors = "|".join(authors)
        books.at[index, "alt_title"] = title
        books.at[index, "alt_author"] = authors
    if index%100==0:
        print(time.time()-start, index)
        start = time.time()
        time.sleep(5)
        if index%1000==0:
            print("SAVE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
            books.to_csv(location_to_save+"items_books_some_isbn.csv")

The Hunger Games
0.30138516426086426 0
SAVE!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
Harry Potter and the Philosopher's Stone
Twilight
To Kill a Mockingbird
The Great Gatsby
The Fault in Our Stars
The Hobbit or There and Back Again
The Catcher in the Rye
Angels & Demons 
Pride and Prejudice
The Kite Runner 
Divergent
Nineteen Eighty-Four
Animal Farm: A Fairy Story
Het Achterhuis: Dagboekbrieven 14 juni 1942 - 1 augustus 1944
Män som hatar kvinnor
Catching Fire
Harry Potter and the Prisoner of Azkaban
 The Fellowship of the Ring
Mockingjay
Harry Potter and the Order of the Phoenix
The Lovely Bones
Harry Potter and the Chamber of Secrets
Harry Potter and the Goblet of Fire
Harry Potter and the Deathly Hallows
The Da Vinci Code
Harry Potter and the Half-Blood Prince
Lord of the Flies 
An Excellent conceited Tragedie of Romeo and Juliet
Gone Girl
The Help
Of Mice and Men 
Memoirs of a Geisha
Fifty Shades of Grey
O Alquimista
The Giver
The Lion, the Witch and the Wardrobe
The

D. Save file.

In [ ]:
books.to_csv(location_to_save+"items_books_some_isbn.csv")

## 5. Analyze results from Google Books API
<b>Goal</b>: To check the structure and completeness of the Google Books information. Also, to compare the author name received by Google Books to the author name already existing in the dataset.

A. Read file.

In [27]:
books = pd.read_csv(location_to_save + "items_books_some_isbn.csv", low_memory = False, index_col=0).fillna("")

B. Check completeness.

In [28]:
no_results_authors = len(books[(books.alt_author=="")].drop_duplicates(subset = "author"))
no_results_books = len(books[(books.alt_author=="")])

In [29]:
print("For", no_results_authors,"unique authors, we couldn't match the ISBN of at least one of their books, out of",len(books.drop_duplicates(subset = "author")), "authors.")
print("For", no_results_books,"books, we couldn't match the ISBN, out of", len(books), "books.")

For 6511 unique authors, we couldn't match the ISBN of at least one of their books, out of 102024 authors.
For 10134 books, we couldn't match the ISBN, out of 271360 books.


C. Compare Book-Crossing with Google Books. We aim to see whether the results match the actual information on the dataset, in particular in terms of author name.

C.1 First estimation: Naive, which means directly comparing the strings of "author" and "alt_author".

In [30]:
first_estimation = len(books[books.author!=books.alt_author])
print("At first glance, we didn't find the same author for", first_estimation,"books out of ",str(len(books))+". However, it needs more processing.")

At first glance, we didn't find the same author for 92052 books out of  271360. However, it needs more processing.


C.2 Second estimation: Simplify author names.

We need a function that:
1. Removes spaces
2. Removes punctuation
3. Makes lowercase

<b>This way, we can more properly compare the strings.</b>

In [251]:
def simplify(name):
    name = name.replace(" ","").translate(str.maketrans('', '', string.punctuation)).lower()
    return name

In [32]:
simplify("Savvina G. Daniil")

'savvinagdaniil'

We create a column called "correct_author" where we replace it with the alt_author if simplified it is the same as the book crossing author. 

In [33]:
books["correct_author"] = ''
for index,row in books.iterrows():
    if row.alt_author!="":
        if simplify(row.author)==simplify(row.alt_author):
            books.at[index, "correct_author"] = row.alt_author

In [34]:
second_estimation = len(books[books.correct_author==""])
print("With the simplifying, we didn't find the same author for", second_estimation,"books out of ",str(len(books))+". However, it needs even more processing.")

With the simplifying, we didn't find the same author for 81731 books out of  271360. However, it needs even more processing.


C.3 Third estimation: Account for multiple authors.

Since in alt_author we collect all authors, we make sure that we consider if any of them (separated by "|") is the same as author.

In [35]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        if row.alt_author!="": # if we got ISBN result
            alt_author = row.alt_author
            bag_of_alt_authors = alt_author.split("|")
            simplified_bag_of_alt_authors = [simplify(x) for x in bag_of_alt_authors]
            author = simplify(row.author)
            for i in range(len(bag_of_alt_authors)):
                if author==simplified_bag_of_alt_authors[i]:
                    books.at[index, "correct_author"] = bag_of_alt_authors[i]

In [36]:
third_estimation = len(books[books.correct_author==""])
print("With accounting for bag of authors, we didn't find the same author for", third_estimation,"books out of ",str(len(books))+". However, it needs even more processing.")

With accounting for bag of authors, we didn't find the same author for 50833 books out of  271360. However, it needs even more processing.


C.4 Fourth estimation: Reverse names.

We noticed that some names are reversed, e.g. Charles Glass and Glass Charles are not considered the same name by our processing. So we add an extra layer: sort the characters of the name. 

In [37]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [sorted(simplify(x)) for x in bag_of_alt_authors]
        author = sorted(simplify(row.author))
        for i in range(len(bag_of_alt_authors)):
            if author==simplified_bag_of_alt_authors[i]:
                books.at[index, "correct_author"] = bag_of_alt_authors[i]

In [38]:
fourth_estimation = len(books[books.correct_author==""])
print("With the sorting trick, we didn't find the same author for", fourth_estimation,"books out of ",str(len(books))+". However, it needs even more processing.")

With the sorting trick, we didn't find the same author for 49731 books out of  271360. However, it needs even more processing.


C.5 Fifth estimation: String similarity. 

At this point, we can no longer rely on simply the literal comparison between strings. We need to introduce a similarity measure between them.

In [39]:
def similarity(a, b):
    return SequenceMatcher(None, a, b).ratio()

In [40]:
similarity("Savvina Daniil", "Savvina G. Daniil"), similarity("Savvina Daniil", "John Williams")

(0.9032258064516129, 0.2962962962962963)

We consider every similarity over 0.7 to be the same name.

In [41]:
for index, row in books.iterrows():
    if row.correct_author=="": # only if we haven't already found the correct author of that book
        alt_author = row.alt_author
        bag_of_alt_authors = alt_author.split("|")
        simplified_bag_of_alt_authors = [sorted(simplify(x)) for x in bag_of_alt_authors]
        author = sorted(simplify(row.author))
        
        max_sim = 0
        max_i = -1000
        
        for i in range(len(bag_of_alt_authors)):
            sim = similarity(author, simplified_bag_of_alt_authors[i])
            if sim>max_sim:
                max_sim = sim
                max_i = i
        
        if max_sim>=0.7:
            books.at[index, "correct_author"] = bag_of_alt_authors[max_i]

In [42]:
fifth_estimation = len(books[books.correct_author==""])
print("With the similarity threshold, we didn't find the same author for", fifth_estimation,"books out of ",str(len(books))+". At this point, it is hard to further progress.")

With the similarity threshold, we didn't find the same author for 22019 books out of  271360. At this point, it is hard to further progress.


In [43]:
print("FINAL SUM UP")
print("Books:                           |", len(books))
print("Books with ISBN results:         |", len(books[books.alt_author!=""]))
print("Books with matching ISBN results:|", len(books[books.correct_author!=""]))

FINAL SUM UP
Books:                           | 271360
Books with ISBN results:         | 261226
Books with matching ISBN results:| 249341


C.6 Save first author from GB. 
We may need to use them when running the viaf API.

In [44]:
books["alt_first_author"] = books.alt_author.apply(lambda x: x.split("|")[0])

In [45]:
books.head()

,ISBN,title,author,year,publisher,alt_title,alt_author,correct_author,alt_first_author
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,Decision in Normandy,Carlo D'Este,Carlo D'Este,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,Flu,Gina Bari Kolata,Gina Bari Kolata,Gina Bari Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,The Mummies of Ürümchi,E. J. W. Barber,E. J. W. Barber,E. J. W. Barber


C.7 Save file

In [46]:
books.to_csv(location_to_save+"items_books_some_isbn.csv")

## 6. Access VIAF API
<b>Goal</b>: To run "correct_author" if it exists to the VIAF(https://viaf.org/) API, so as to get a VIAF id. 

Note: For now, we are only running it for the authors that have a "correct_author" name, i.e., the ones for whom their BC name matched their GB name based on the previous analysis.

In [47]:
books = pd.read_csv(location_to_save+"items_books_some_isbn.csv", low_memory = False, index_col=0).fillna("")

A. Function to access the API.

In [43]:
def get_viaf_entry(author_name):
    #print(author_name)
    viaf = ViafAPI()
    viaf_id = ""
    viaf_name = ""
    potential_entries = viaf.suggest(author_name)[:5]
    if len(potential_entries)==0:
        print("No results!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")
    for entry in potential_entries:
        if entry["nametype"]=="personal":
            #print(entry["term"])
            try:
                viaf_id = entry["viafid"]
                viaf_name = entry["term"]
                #print("We found viaf_id!")
            except:
                print(author_name + " has no Viaf ID.")
            break
        print("ISSUE! We didn't find for", author_name)
    return viaf_id, viaf_name

B. Initialize the entries.

In [55]:
books["viaf_id"] = ""
books["viaf_name"] = ""

C. Define unique authors.

The purpose is to not rerun the API again for the same "correct author" name.

In [49]:
definitive_unique_authors = books[books.correct_author!=""].drop_duplicates(subset="correct_author")

D. Run the requests.

In [ ]:
start = time.time()
i=0
for index, row in definitive_unique_authors.iloc[86000:].iterrows():
    if row.viaf_id=="":
        author = row.correct_author
        viaf_id, viaf_name = get_viaf_entry(author)
        if viaf_id == "":
            print("try normal author name")
            viaf_id, viaf_name = get_viaf_entry(row.author)
        books.at[index, "viaf_id"] = viaf_id
        books.at[index, "viaf_name"] = viaf_name
        definitive_unique_authors.at[index, "viaf_id"] = viaf_id
        definitive_unique_authors.at[index, "viaf_name"] = viaf_name
    if i%500==0:
        print(index, time.time()-start)
        books.to_csv(location_to_save+"items_books_after_viaf.csv")
    i+=1

E. Save file.

In [75]:
books.to_csv(location_to_save+"items_books_after_viaf.csv")

## 7. Complete the VIAF enriched dataset
<b>Goal</b>: Given that we dropped duplicates for "correct_author", we now need to fill the entire books dataset with the VIAF ids we got.

A. Read file.

In [50]:
books = pd.read_csv(location_to_save+"items_books_after_viaf.csv", dtype = object, index_col=0).fillna("")

B. Keep books with matching ISBN results and then authors with viaf results.

In [164]:
books_with_matching_ISBN_results = books[books.correct_author!=""]

In [165]:
authors_with_viaf = books_with_matching_ISBN_results[["correct_author", "viaf_id", "viaf_name"]].loc[books_with_matching_ISBN_results.viaf_id!=""]

In [166]:
books_with_matching_ISBN_results = books_with_matching_ISBN_results.drop(["viaf_id", "viaf_name"], axis =1)

C. Keep books with mathcing ISBN results and viaf results.

In [167]:
books_with_matching_ISBN_results_and_viaf = pd.merge(books_with_matching_ISBN_results.reset_index(), authors_with_viaf, on="correct_author").set_index("index")

D. Fill the books dataset with these results.

In [168]:
books.loc[books_with_matching_ISBN_results_and_viaf.index] = books_with_matching_ISBN_results_and_viaf

E. Sum up.

In [183]:
print("FINAL SUM UP                              | Found  | Remaining")
print("------------------------------------------|--------|----------")
print("Books:                                    |", len(books),"|", len(books) - len(books))
print("Books with ISBN results:                  |", len(books[books.alt_author!=""]),"|", len(books[books.alt_author==""]))
print("Books with matching ISBN results:         |", len(books[books.correct_author!=""]),"|", len(books[books.correct_author==""]))
print("Books with matching ISBN and VIAF results:|", len(books[books.viaf_id!=""]),"|", len(books[books.viaf_id==""]))

FINAL SUM UP                              | Found  | Remaining
------------------------------------------|--------|----------
Books:                                    | 271360 | 0
Books with ISBN results:                  | 261226 | 10134
Books with matching ISBN results:         | 249341 | 22019
Books with matching ISBN and VIAF results:| 244457 | 26903


F. Save file.

In [170]:
books.to_csv(location_to_save+"items_books_after_viaf.csv")

## 8. Rerun VIAF API for books without matching ISBN results
There are ~5K books with matcing authors for whom we didn't find VIAF id. However, we didn't try books without matching authors, or even any Google Books results at all.

<b>Goal</b>: To run again the VIAF API, this time for the remaining authors (i.e., those for whom we don't have the "correct_author" name).

A. Read file.

In [10]:
books = pd.read_csv(location_to_save+"items_books_after_viaf.csv", dtype = object, index_col=0).fillna("")

B. Define authors to run: unique in terms of combination of author and GB first author.

In [11]:
unique_unfilled_authors = books[books.correct_author==""].drop_duplicates(["author", "alt_first_author"])

In [12]:
print("There are",len(unique_unfilled_authors), "unique authors for whom we didn't run the VIAF API.")

There are 15732 unique authors for whom we didn't run the VIAF API.


C. Run VIAF again.

In [ ]:
start = time.time()
i=0
for index, row in unique_unfilled_authors.iterrows():
    if books.at[index,"viaf_id"]=="":
        author = row.author
        alt_first_author = row.alt_first_author
        viaf_id_author, viaf_name_author = get_viaf_entry(author)
        if alt_first_author!="":
            viaf_id_alt_first_author, viaf_name_alt_first_author = get_viaf_entry(alt_first_author)
        else:
            viaf_id_alt_first_author, viaf_name_alt_first_author = "", ""
        books.at[index, "viaf_id"] = viaf_id_author+"||"+viaf_id_alt_first_author
        books.at[index, "viaf_name"] = viaf_name_author+"||"+viaf_name_alt_first_author
        unique_unfilled_authors.at[index, "viaf_id"] = viaf_id_author+"||"+viaf_id_alt_first_author
        unique_unfilled_authors.at[index, "viaf_name"] = viaf_name_author+"||"+viaf_name_alt_first_author
        if i%500==0:
            print(index, time.time()-start)
            books.to_csv(location_to_save+"items_books_after_viaf.csv")
        i+=1

D. Save file.

In [160]:
books.to_csv(location_to_save+"items_books_after_viaf.csv")

## 9. Complete VIAF enriched dataset (again)
<b>Goal</b>: Given that we dropped duplicates for combination of "author" and "alt_first_author", we now need to fill the entire books dataset with the viaf ids we got.

A. Read file.

In [161]:
books = pd.read_csv(location_to_save+"items_books_after_viaf.csv", dtype = object, index_col=0).fillna("")

B. Keep books without matching ISBN results and then authors with viaf results.

In [162]:
books_without_matching_ISBN_results = books.loc[books.correct_author==""]

In [163]:
authors_with_viaf = books_without_matching_ISBN_results[["author","alt_first_author","viaf_id","viaf_name"]].loc[books.viaf_id!=""]

In [164]:
books_without_matching_ISBN_results = books_without_matching_ISBN_results.drop(["viaf_id","viaf_name"], axis=1)

C. Keep books with mathcing ISBN results and viaf results.

In [165]:
books_without_matching_ISBN_results_and_with_viaf = pd.merge(books_without_matching_ISBN_results.reset_index(), authors_with_viaf, on=["author","alt_first_author"]).set_index("index")

D. Fill the books dataset with these results.

In [166]:
books.loc[books_without_matching_ISBN_results_and_with_viaf.index] = books_without_matching_ISBN_results_and_with_viaf

E. Sum up.

In [167]:
print("FINAL SUM UP                                                       | Found  | Remaining")
print("-------------------------------------------------------------------|--------|----------")
print("Books:                                                             |", len(books),"|", len(books) - len(books))
print("Books without matching ISBN results:                               |", len(books[books.correct_author==""])," |", len(books[books.correct_author!=""]))
print("Books without matching ISBN results but with VIAF results:         |", len(books[(books.correct_author=="")&(books.viaf_id!="||")])," |", len(books)-len(books[(books.correct_author=="")&(books.viaf_id!="||")]))

FINAL SUM UP                                                       | Found  | Remaining
-------------------------------------------------------------------|--------|----------
Books:                                                             | 271360 | 0
Books without matching ISBN results:                               | 22019  | 249341
Books without matching ISBN results but with VIAF results:         | 20108  | 251252


F. Save file.

In [168]:
books.to_csv(location_to_save+"items_books_after_viaf.csv")

## 10. Solve double VIAF ids.
For the cases where there wasn't matching ISBN results, we ran both the BC author and the GB author in the VIAF API. This means that sometimes we got duplicated results for VIAF id.

<b>Goal</b>: To resolve this issue by dropping a VIAF id when needed. 

A. Read file.

In [33]:
books = pd.read_csv(location_to_save+"items_books_after_viaf.csv", dtype = object, index_col=0).fillna("")

B. Consider only books without matching ISBN results but with VIAF results (because this is the only case where we might have got duplicate ids).

In [34]:
relevant_books = books[(books.viaf_id.str.contains("||",regex = False))&(books.viaf_id!="||")].copy()

C. Consider both retrieved viaf results.

In [271]:
relevant_books["viaf_id_1"] = relevant_books.viaf_id.apply(lambda x: x.split("||")[0])
relevant_books["viaf_id_2"] = relevant_books.viaf_id.apply(lambda x: x.split("||")[-1])

relevant_books["viaf_name_1"] = relevant_books.viaf_name.apply(lambda x: x.split("||")[0])
relevant_books["viaf_name_2"] = relevant_books.viaf_name.apply(lambda x: x.split("||")[-1])

In [272]:
len(relevant_books), len(books)

(20108, 271360)

D. Consider the following cases:
1. If the viaf ids are the same, keep the common one.
2. If one of the viaf ids is "", keep the other one.

In [273]:
for index, row in relevant_books.iterrows():
    viaf_id_1 = row.viaf_id_1
    viaf_id_2 = row.viaf_id_2
    if viaf_id_1 == viaf_id_2: # if they are the same
        relevant_books.at[index, "viaf_id"] = viaf_id_1
        relevant_books.at[index, "viaf_name"] = row.viaf_name_1
    elif viaf_id_1 == "":
        relevant_books.at[index, "viaf_id"] = viaf_id_2
        relevant_books.at[index, "viaf_name"] = row.viaf_name_2
    elif viaf_id_2 == "":
        relevant_books.at[index, "viaf_id"] = viaf_id_1
        relevant_books.at[index, "viaf_name"] = row.viaf_name_1

In [274]:
len(relevant_books[(relevant_books.viaf_id.str.contains("||",regex = False))])

5403

So 5403 books yielded two different valid viaf results. 

E. Consider the following cases:
1. If the GB author name and the viaf name are the same, keep the GB author.
2. In any other case, keep the BC author. 

In [275]:
less_relevant_books = relevant_books[(relevant_books.viaf_id.str.contains("||",regex = False))]

In [276]:
for index, row in less_relevant_books.iterrows():
    viaf_id_1 = row.viaf_id_1
    viaf_id_2 = row.viaf_id_2
    
    viaf_name_1 = row.viaf_name_1
    viaf_name_2 = row.viaf_name_2
    
    check_1 = (sorted(simplify(viaf_name_1.rsplit(',', 1)[0])) == sorted(simplify(row.author)))
    check_2 = (sorted(simplify(viaf_name_2.rsplit(',', 1)[0])) == sorted(simplify(row.alt_first_author)))
    
    if (check_2 == True) & (check_1 == False):
        less_relevant_books.at[index, "viaf_id"] = viaf_id_2
        less_relevant_books.at[index, "viaf_name"] = row.viaf_name_2
        
    else:
        less_relevant_books.at[index, "viaf_id"] = viaf_id_1
        less_relevant_books.at[index, "viaf_name"] = row.viaf_name_1

In [277]:
relevant_books.loc[less_relevant_books.index] = less_relevant_books

F. Replace the book entries with the updated.

In [278]:
books.loc[relevant_books.index] = relevant_books.drop(["viaf_id_1", "viaf_id_2", "viaf_name_1", "viaf_name_2"], axis=1)

In [284]:
# Replace the || entries to reach a final dataset.
books.viaf_id = books.viaf_id.apply(lambda x: "" if x == "||" else x)
books.viaf_name = books.viaf_name.apply(lambda x: "" if x == "||" else x)

G. Save file.

In [287]:
books.to_csv(location_to_save+"final_items_books.csv")

## 11. Match Book-Crossing with WikiData
<b>Goal</b>: To finally match BC+ with WikiData. We will do that using the VIAF id we obtained at the previous steps.

A. Read files.

In [25]:
# books-crossing + Google Books + VIAF
books = pd.read_csv(location_to_save+"final_items_books.csv", dtype = object, index_col=0).fillna("")    

# WikiData
with open(location_to_save + "items_with_viaf_wikidata.pkl","rb") as f:
    wd_results =  pkl.load(f)

In [26]:
wd_results.head()

,QID,viaf_id
0,Q270705,16565
1,Q289693,135150789
2,Q481146,14775085
3,Q507746,6650
4,Q568833,158264550


B. Merge datasets on VIAF id.

In [27]:
merged_dataset = pd.merge(books.reset_index(), wd_results, on="viaf_id").set_index("index")

In [28]:
len(merged_dataset)

187775

In [29]:
len(merged_dataset[merged_dataset.duplicated(subset = "ISBN")])

88

There are 88 items that are duplicated! In the sense that either two QIDs are connected to one VIAF id, or the other way around. We will deal with it at future step.

C. Save meged dataset.

In [31]:
merged_dataset.to_csv(location_to_save+"final_merged_dataset.csv")

## 12. Call for WikiData properties
<b>Goal</b>: To fill the dataset with author information. We use WikiData QID to access the WikiData entry of each author using the WikiData API.

A. Read file.

In [3]:
books = pd.read_csv(location_to_save+"final_merged_dataset.csv", dtype = object, index_col=0).fillna("")    

B. Set properties that we are looking for.

In [4]:
properties = {"sexuality":"P91",
             "country":"P27",
             "language":"P6886",
             "religion":"P140",
             "gender":"P21",
             "ethnicity":"P172",
             "birthyear":"P569",
             "deathyear":"P570"}

C. Keep unique authors.

In [5]:
unique_authors = books.drop_duplicates(["viaf_id","QID"]).copy()

In [6]:
for k,v in properties.items():
    unique_authors[k]=""
unique_authors["label"] = ""

In [7]:
len(unique_authors)

40529

D. Run WikiData API.

In [ ]:
start = time.time()
i=0
for index,row in unique_authors.iloc[7860:].iterrows():
    author_qid = row["QID"]
    author_dict = get_entity_dict_from_api(author_qid)
    
    for prop, qid in properties.items():
        if prop=="birthyear" or prop=="deathyear":
            try:
                value = [x["mainsnak"]["datavalue"]["value"]["time"] for x in author_dict["claims"][qid]]
            except:
                value = ""
        else:
            try:
                value = [x["mainsnak"]["datavalue"]["value"]["id"] for x in author_dict["claims"][qid]]
            except:
                value = ""
        unique_authors.at[index,prop] = str(value)
    
    try:
        label = author_dict["labels"]["en"]["value"] # author name in WikiData
    except:
        label = ""
    unique_authors.at[index,"label"] = label
    
    if i%100==0:
        print("100 is gone!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!", index)
        print(time.time()-start)
    if i%500==0:
        unique_authors.to_csv(location_to_save + 'unique_authors_with_properties.csv')
    i+=1

E. Save file.

In [33]:
unique_authors.to_csv(location_to_save + 'unique_authors_with_properties.csv')

## 13. WikiData properties to labels
Most of the properties we gathered have QID values rather than explainable ones. 
<b>Goal</b>: To use WikiData API again to turn the QID values of the properties into readable labels.

A. Read files.

In [37]:
unique_authors = pd.read_csv(location_to_save+"unique_authors_with_properties.csv", dtype = object, index_col=0).fillna("")    

B. Process Qvalues.

In [38]:
relevant_columns = list(unique_authors.columns[12:-3])

In [40]:
relevant_columns # the ones that have QID values

['sexuality', 'country', 'language', 'religion', 'gender', 'ethnicity']

In [43]:
unique_authors[relevant_columns].head()

,sexuality,country,language,religion,gender,ethnicity
index,,,,,,
1,,['Q16'],,,['Q6581097'],
2,,"['Q30', 'Q38']",,,['Q6581097'],
3,,['Q30'],,,['Q6581072'],
4,,['Q30'],['Q1860'],,['Q6581072'],
5,,['Q30'],['Q1860'],['Q93191'],['Q6581072'],


In [44]:
for col in relevant_columns: # Change the syntax of the entry: comma separated 
    unique_authors[col] = unique_authors[col].apply(lambda d: re.sub(r'\W+', ',', d).strip(","))

In [45]:
unique_authors[relevant_columns].head()

,sexuality,country,language,religion,gender,ethnicity
index,,,,,,
1,,Q16,,,Q6581097,
2,,"Q30,Q38",,,Q6581097,
3,,Q30,,,Q6581072,
4,,Q30,Q1860,,Q6581072,
5,,Q30,Q1860,Q93191,Q6581072,


In [47]:
unique_authors.gender.unique() # example of how the values look like now

array(['Q6581097', 'Q6581072', '', 'Q189125', 'Q48270,Q6581072',
       'Q1052281', 'Q48270', 'Q2449503', 'Q6581097,Q6581072',
       'Q48270,Q96000630', 'Q18116794,Q6581097', 'Q6581072,Q6581097',
       'Q1052281,Q6581072', 'Q1097630', 'Q1097630,Q48270',
       'Q6581072,Q1097630', 'Q27679684,Q6581097'], dtype=object)

C. Create property dict. We basically collect the unique Qvalues for every property.

In [48]:
property_labels = {}
for col in relevant_columns:
    property_labels[col] = []
    unique_values = unique_authors[col].unique()
    for value in unique_values:
        if value!="":
            values = value.split(",")
            for val in values:
                if val not in property_labels[col]:
                    property_labels[col].append(val)

In [50]:
property_labels["gender"] # example

['Q6581097',
 'Q6581072',
 'Q189125',
 'Q48270',
 'Q1052281',
 'Q2449503',
 'Q96000630',
 'Q18116794',
 'Q1097630',
 'Q27679684']

D. Turn QID to readable label.

In [175]:
start = time.time()
for col in relevant_columns:
    new_vals = []
    for val in property_labels[col]:
        prop_dict = get_entity_dict_from_api(val)
        try:
            new_val = prop_dict["labels"]["en"]["value"]      
        except:
            # take the first if english doesn't exist
            print(prop_dict["labels"])
            new_val = list(prop_dict["labels"].values())[0]["value"]
        new_vals.append(new_val)
    property_labels[col].append(new_vals)
    print(time.time() - start)
    print(col)

3.39420223236084
sexuality
204.8985240459442
country
240.67512369155884
language
{'de': {'language': 'de', 'value': 'Surmang-Kagyü'}}
294.4538872241974
religion
297.62130784988403
gender
{'be': {'language': 'be', 'value': 'Чэхі, племя'}, 'ru': {'language': 'ru', 'value': 'Чехи (племя)'}, 'cs': {'language': 'cs', 'value': 'Čechové'}, 'ro': {'language': 'ro', 'value': 'cehi'}}
{'es': {'language': 'es', 'value': 'Wasco'}, 'ca': {'language': 'ca', 'value': 'wasco'}, 'vec': {'language': 'vec', 'value': 'wasco'}}
{'ru': {'language': 'ru', 'value': 'евреи в Румынии'}, 'eo': {'language': 'eo', 'value': 'rumanaj judoj'}}
373.84655117988586
ethnicity


In [176]:
for col in property_labels.keys():
    vals = property_labels[col]
    rang = len(vals) - 1
    label_list = vals[-1]
    label_dict = {}
    for i in range(rang):
        label_dict[vals[i]] = label_list[i]
    property_labels[col] = label_dict
    print(col)

sexuality
country
language
religion
gender
ethnicity


In [177]:
print("Unique values per property")
for column in property_labels.keys():
    print(column+": "+str(len(property_labels[column].keys())))


Unique values per property
sexuality: 8
country: 377
language: 90
religion: 187
gender: 10
ethnicity: 260


E. Save property dict.

In [178]:
with open(location_to_save+"property_dict.pickle", "wb") as f: # naming it pickle so it's in github
    pkl.dump(property_labels,f)

F. Replace values in the dataframe itself.

In [179]:
for col in relevant_columns:
    series = unique_authors[col]
    new_series = []
    for item in series:
        if item!="":
            items = item.split(",")
            new_items = []
            for it in items:
                new_it = property_labels[col][it]
                new_items.append(new_it)
            new_items = ",".join(new_items)
        else: 
            new_items = ""
        new_series.append(new_items)
    unique_authors[col] = new_series
    print(col)

sexuality
country
language
religion
gender
ethnicity


In [180]:
unique_authors.gender.unique()

array(['male', 'female', '', 'transgender person', 'non-binary,female',
       'transgender female', 'non-binary', 'transgender male',
       'male,female', 'non-binary,X-gender', 'genderfluid,male',
       'female,male', 'transgender female,female', 'intersex',
       'intersex,non-binary', 'female,intersex', 'transfeminine,male'],
      dtype=object)

G. Process birth and deathyear.

In [51]:
unique_authors.birthyear[1]

"['+1937-03-04T00:00:00Z']"

In [52]:
for index, row in unique_authors[["birthyear", "deathyear"]].iterrows():
    b = row.birthyear
    d = row.deathyear
    if b!="":
        if b[0]=="[":
            bs = b.split("['")
            b_entry = bs[1]
            new_b = b_entry[:5]
        else:
            new_b = b[:5]
        
        unique_authors.at[index, "birthyear"] = new_b
    
    if d!="":
        if d[0]=="[":
            ds = d.split("['")
            d_entry = ds[1]
            new_d = d_entry[:5]
        else:
            new_d = d[:5]
        unique_authors.at[index, "deathyear"] = new_d

In [53]:
unique_authors.birthyear[1]

'+1937'

H. Save file.

In [192]:
unique_authors.to_csv(location_to_save + 'unique_authors_with_readable_properties.csv')

## 14. Solve the double VIAF id - Qid problem
<b>Goal</b>: To investigate why there is not a 1-1 relationship between VIAF ids and Qids and solve the issue.

A. Read file: it's the unique authors with filled readable properties.

In [54]:
unique_authors = pd.read_csv(location_to_save+"unique_authors_with_readable_properties.csv", dtype = object, index_col=0).fillna("")    

In [62]:
duplicate_viaf_ids = unique_authors[unique_authors.duplicated("viaf_id", keep = False)].sort_values(by = "viaf_id")

In [63]:
len(duplicate_viaf_ids)

72

In [64]:
duplicate_viaf_ids.columns[1:]

Index(['title', 'author', 'year', 'publisher', 'alt_title', 'alt_author',
       'correct_author', 'alt_first_author', 'viaf_id', 'viaf_name', 'QID',
       'sexuality', 'country', 'language', 'religion', 'gender', 'ethnicity',
       'birthyear', 'deathyear', 'label'],
      dtype='object')

B. Remove duplicate viaf id rows if they are the same in terms of every property apart from qid. 

In [65]:
len(duplicate_viaf_ids.drop_duplicates(subset=list(duplicate_viaf_ids.columns[1:])))

72

It's the same number, which means that the double qids are actually mistakenly twice put people.

In [66]:
unique_viaf_ids = duplicate_viaf_ids.drop_duplicates(subset=list(duplicate_viaf_ids.columns[1:]))

In [67]:
viaf_id_to_QID_map = dict(zip(unique_viaf_ids.viaf_id, unique_viaf_ids.QID))

In [68]:
duplicate_viaf_ids["QID"] = duplicate_viaf_ids["viaf_id"].map(viaf_id_to_QID_map)

In [78]:
duplicate_viaf_ids = duplicate_viaf_ids.drop_duplicates()

In [79]:
unique_authors.loc[list(duplicate_viaf_ids.index)] = duplicate_viaf_ids

In [82]:
unique_authors = unique_authors.drop_duplicates()

C. Save file.

In [84]:
unique_authors.to_csv(location_to_save + 'unique_authors_with_readable_properties.csv')

D. Check the duplicate Qids.

In [85]:
duplicate_Qids = unique_authors[unique_authors.duplicated("QID", keep = False)].sort_values(by = "QID")

In [90]:
duplicate_Qids[["author", "alt_author", "correct_author", "viaf_id", "viaf_name", "QID", "label"]].head(10)

,author,alt_author,correct_author,viaf_id,viaf_name,QID,label
index,,,,,,,
138202,M Masters,M. Masters,M. Masters,92725843,M Masters,Q10363150,Robert Alexander
58684,R. D. Zimmerman,R. D. Zimmerman,R. D. Zimmerman,51809100,R. D Zimmerman,Q10363150,Robert Alexander
206069,David A Myers,David A. Myers,David A. Myers,85444993,David A Myers,Q1174497,David Myers
27195,David Myers,David Myers,David Myers,54210999,David Myers American psychologist,Q1174497,David Myers
159754,Three Initiates,Three Initiates|Three Initiates Staff,Three Initiates,31992123,Three Initiates,Q1335019,William Walker Atkinson
204045,Yogi Ramacharaka,Yogi Ramacharaka,Yogi Ramacharaka,103211604,Yogi Rāmacharaka,Q1335019,William Walker Atkinson
29109,Karen Keast,Karen Keast,Karen Keast,270879607,Karen Keast,Q17409349,Sandra Canfield
10960,Sandra Canfield,Sandra Canfield,Sandra Canfield,7411024,Sandra Canfield,Q17409349,Sandra Canfield
5730,Caroline Arnold,Judith Arnold|Rachel Lindsay,Rachel Lindsay,88010581,"Rachel Lindsay, 1926-2014",Q3161936,Roberta Leigh


By manually checking, it looks like they are pen names. In this case, we can just drop because it doesn't matter.

In [91]:
unique_Qids = duplicate_Qids.drop_duplicates(subset=list(duplicate_Qids.columns[1:]))

In [92]:
QID_to_viaf_id_map = dict(zip(unique_Qids.QID, unique_Qids.viaf_id))

In [93]:
duplicate_Qids["viaf_id"] = duplicate_Qids["QID"].map(QID_to_viaf_id_map)

In [94]:
duplicate_Qids = duplicate_Qids.drop_duplicates()

In [95]:
unique_authors.loc[list(duplicate_Qids.index)] = duplicate_Qids

/export/scratch2/home/savvina/new_environment/lib64/python3.6/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/export/scratch2/home/savvina/new_environment/lib64/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [99]:
unique_authors = unique_authors.drop_duplicates(subset = ["QID", "viaf_id"])

In [103]:
len(unique_authors)

40471

E. Save file.

In [104]:
unique_authors.to_csv(location_to_save + 'unique_authors_with_readable_properties.csv')

## 15. Create "Final" Datasets
<b>Goal</b>: To fill the datasets with the information collected in unique_authors.

A. Read file: it's the unique authors with filled readable properties.

In [3]:
unique_authors = pd.read_csv(location_to_save+"unique_authors_with_readable_properties.csv", dtype = object, index_col=0).fillna("")    

In [4]:
unique_authors = unique_authors[["QID", "viaf_id","sexuality","country",
                                "language","religion","gender","ethnicity",
                                "birthyear","deathyear","label"]]

B. Fill Wikidata books.

In [5]:
wikidata_books = pd.read_csv(location_to_save+"final_merged_dataset.csv", low_memory=False, index_col=0).fillna("")

In [6]:
wikidata_books = wikidata_books.drop_duplicates(["QID","ISBN"]).drop_duplicates(["viaf_id","ISBN"])

In [7]:
wikidata_books_with_properties = pd.merge(wikidata_books, unique_authors, on=["viaf_id","QID"], how="left").fillna("")

C. Fill entire books.

In [8]:
books = pd.read_csv(location_to_save+"final_items_books.csv", dtype = object, index_col=0).fillna("")

In [9]:
books_with_properties = pd.merge(books,wikidata_books_with_properties, on=list(books.columns), how="left").fillna("")

In [10]:
books_with_properties.head()

,ISBN,title,author,year,publisher,alt_title,alt_author,correct_author,alt_first_author,viaf_id,...,QID,sexuality,country,language,religion,gender,ethnicity,birthyear,deathyear,label
0,0195153448,Classical Mythology,Mark P. O. Morford,2002,Oxford University Press,Classical Mythology,Mark P. O. Morford|Robert J. Lenardon,Mark P. O. Morford,Mark P. O. Morford,92220559,...,,,,,,,,,,
1,0002005018,Clara Callan,Richard Bruce Wright,2001,HarperFlamingo Canada,Clara Callan,Richard Bruce Wright,Richard Bruce Wright,Richard Bruce Wright,64022406,...,Q7323887,,Canada,,,male,,+1937,+2017,Richard B. Wright
2,0060973129,Decision in Normandy,Carlo D'Este,1991,HarperPerennial,Decision in Normandy,Carlo D'Este,Carlo D'Este,Carlo D'Este,97755576,...,Q2939184,,"United States of America,Italy",,,male,,+1936,+2020,Carlo D'Este
3,0374157065,Flu: The Story of the Great Influenza Pandemic...,Gina Bari Kolata,1999,Farrar Straus Giroux,Flu,Gina Bari Kolata,Gina Bari Kolata,Gina Bari Kolata,111861625,...,Q1524875,,United States of America,,,female,,+1948,,Gina Kolata
4,0393045218,The Mummies of Urumchi,E. J. W. Barber,1999,W. W. Norton &amp; Company,The Mummies of Ürümchi,E. J. W. Barber,E. J. W. Barber,E. J. W. Barber,91244635,...,Q5363702,,United States of America,English,,female,,+1940,,Elizabeth Wayland Barber


In [11]:
len(books), len(books_with_properties)

(271360, 271360)

D. Fill entire ratings.

In [18]:
ratings = pd.read_csv("data/ratings_books.csv", dtype = object).fillna("")

In [19]:
len(ratings)

1149780

In [21]:
ratings_with_properties = pd.merge(books_with_properties, ratings, on = "ISBN", how="right").fillna("")

In [22]:
len(ratings_with_properties)

1149780

D. Fill fairbooks ratings dataset.

In [24]:
fairbook_ratings = pd.read_csv(location_to_save+"fairbook_ratings.csv", dtype = object, index_col=0).fillna("")    

In [25]:
len(fairbook_ratings)

88552

In [27]:
fairbook_ratings_with_properties = pd.merge(fairbook_ratings, books_with_properties, on = "ISBN", how = "left").fillna("")

In [28]:
len(fairbook_ratings_with_properties)

88552

In [30]:
fairbook_ratings_with_properties.head()

,User-ID,ISBN,Book-Rating,title,author,year,publisher,alt_title,alt_author,correct_author,...,QID,sexuality,country,language,religion,gender,ethnicity,birthyear,deathyear,label
0,276847,3426029553,8,,,,,,,,...,,,,,,,,,,
1,276847,3551551677,10,Harry Potter und der Stein der Weisen,Joanne K. Rowling,1999,Carlsen Verlag GmbH,Harry Potter und der Stein der Weisen,Joanne K. Rowling,Joanne K. Rowling,...,Q34660,,United Kingdom,English,Anglicanism,female,English people,+1965,,J. K. Rowling
2,276847,3551551685,10,Harry Potter und die Kammer des Schreckens,Joanne K. Rowling,2000,Carlsen Verlag GmbH,Harry Potter und die Kammer des Schreckens (Ha...,J.K. Rowling,J.K. Rowling,...,Q34660,,United Kingdom,English,Anglicanism,female,English people,+1965,,J. K. Rowling
3,276847,3551551693,10,Harry Potter und der Gefangene von Azkaban,J. K. Rowling,1999,Carlsen Verlag GmbH,Harry Potter und der Gefangene von Askaban (Ha...,J.K. Rowling,J.K. Rowling,...,Q34660,,United Kingdom,English,Anglicanism,female,English people,+1965,,J. K. Rowling
4,276847,3551551936,10,Harry Potter Und Der Feuerkelch,Joanne K. Rowling,1999,Carlsen Verlag GmbH,Harry Potter und der Feuerkelch (Harry Potter 4),J.K. Rowling,J.K. Rowling,...,Q34660,,United Kingdom,English,Anglicanism,female,English people,+1965,,J. K. Rowling


E. Fill fairbooks item dataset.

In [31]:
fairbook_ISBNs = fairbook_ratings_with_properties.ISBN.unique()

In [32]:
len(fairbook_ISBNs)

6921

In [33]:
fairbook_books_with_properties = books_with_properties[books_with_properties.ISBN.isin(fairbook_ISBNs)]

F. Save all files!

In [34]:
wikidata_books_with_properties.to_csv(location_to_save + 'wikipedia_books_with_readable_properties.csv')
books_with_properties.to_csv(location_to_save + 'items_books_with_readable_properties.csv')
fairbook_ratings_with_properties.to_csv(location_to_save + 'fairbook_ratings_with_readable_properties.csv')
ratings_with_properties.to_csv(large_location_to_save + 'entire_ratings_with_readable_properties.csv')
fairbook_books_with_properties.to_csv(location_to_save + 'fairbook_books_with_readable_properties.csv')

## 16. FIlter out unwanted "wrong" ISBNs
Some of the ratings are of books whose ISBN is not in the books dataset. 

<b>Goal</b>: To drop them.

A. Read files.

In [11]:
entire_books = pd.read_csv(location_to_save+"items_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
entire_ratings = pd.read_csv(large_location_to_save+"entire_ratings_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
fairbook_books = pd.read_csv(location_to_save+"fairbook_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
fairbook_ratings = pd.read_csv(location_to_save+"fairbook_ratings_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")

B. Check the ones not in books.

In [12]:
all_ISBNs = list(entire_books.ISBN)

C. Remove the ones not in books.

In [14]:
fairbook_ratings_correct_ISBN = fairbook_ratings[fairbook_ratings.ISBN.isin(all_ISBNs) == True]

In [15]:
entire_ratings_correct_ISBN = entire_ratings[entire_ratings.ISBN.isin(all_ISBNs) == True]

D. Save files.

In [8]:
fairbook_ratings_correct_ISBN.to_csv(location_to_save + 'fairbook_ratings_with_readable_properties_filtered_ISBNs.csv')
entire_ratings_correct_ISBN.to_csv(large_location_to_save + 'entire_ratings_with_readable_properties_filtered_ISBNs.csv')

In [9]:
len(fairbook_ratings_correct_ISBN), len(fairbook_ratings)

(86782, 88552)

In [10]:
len(entire_ratings_correct_ISBN), len(entire_ratings)

(1031136, 1149780)

## 17. Remove duplicate books
So far, we distinguish between books by using their ISBN. However, it might be that the same books have different ISBNs due to a differnet publisher, year and country of publication.

<b>Goal</b>: To group ISBNs based on common title and author, and then remove duplicate books and ratings.

A. Read files.

In [66]:
entire_books = pd.read_csv(location_to_save+"items_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
fairbook_books = pd.read_csv(location_to_save+"fairbook_books_with_readable_properties.csv", low_memory=False, index_col=0).fillna("")
entire_ratings = pd.read_csv(large_location_to_save+"entire_ratings_with_readable_properties_filtered_ISBNs.csv", low_memory=False, index_col=0).fillna("")
fairbook_ratings = pd.read_csv(location_to_save+"fairbook_ratings_with_readable_properties_filtered_ISBNs.csv", low_memory=False, index_col=0).fillna("")

B. Relevant functions.

In [67]:
def remove_redundant_books(author_df): # for an author create a dictionary with unique ISBN titles.
    author_df.alt_title = author_df.alt_title.apply(lambda x: simplify(x)) #simplify google books author name
    unique_titles = author_df.alt_title.unique() #unique google books titles
    author_df_alt = author_df.copy() 
    for unique_title in unique_titles: #for every unique title
        first_ISBN = author_df.loc[author_df.alt_title == unique_title].iloc[0].ISBN # keep the first appearence ISBN
        author_df_alt.loc[author_df_alt.alt_title == unique_title, "ISBN_alt"] = first_ISBN # ISBN_alt is this first ISBN
        ISBN_dict = dict(zip(author_df_alt.ISBN, author_df_alt.ISBN_alt)) # every ISBN connected to this title is mapped to the first ISBN
    return(ISBN_dict)

In [68]:
def simplify(name):
    name = name.replace(" ","").translate(str.maketrans('', '', string.punctuation)).lower()
    return name

C. Create big ISBN dictionary where each group of ISBNs that refer to the same book are mapped to a representative one.

In [69]:
entire_books["ISBN_alt"] = ""

In [70]:
unique_authors = entire_books[["QID", "viaf_id"]].drop_duplicates()

In [ ]:
start = time.time()
large_ISBN_dict = {}
for index, row in unique_authors.iterrows():
    viaf_id = row.viaf_id
    QID = row.QID
    author_df = entire_books[(entire_books.QID==QID)&(entire_books.viaf_id==viaf_id)][["ISBN", 
                                                                                       "title",
                                                                                      "alt_title",
                                                                                      "ISBN_alt"]]
    ISBN_dict = remove_redundant_books(author_df)
    large_ISBN_dict.update(ISBN_dict)
    del author_df
    del ISBN_dict
    if index%500==0:
        print("500 is gone!!!!!!!!!", index)
        print(time.time()-start)
        start = time.time()
        with open(location_to_save+'ISBN_dict.pkl', 'wb') as handle:
            pkl.dump(large_ISBN_dict, handle)

D. Fill ISBN_alt for all datasets with the appropriate representative.

Book datasets (entire & fairbook).

In [ ]:
for index, row in entire_books.iterrows():
    if row.alt_title!="":  
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        entire_books.at[index, "ISBN_alt"] = ISBN_alt

In [ ]:
fairbook_books["ISBN_alt"] = fairbook_books["ISBN"]
for index, row in fairbook_books.iterrows():
    if row.alt_title!="":
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        fairbook_books.at[index, "ISBN_alt"] = ISBN_alt

Rating datasets (entire & fairbook).

In [ ]:
for index, row in entire_ratings.iterrows():
    if row.alt_title!="":
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        entire_ratings.at[index, "ISBN"] = ISBN_alt
entire_ratings_correct = entire_ratings.drop_duplicates(["ISBN", "User-ID"])

In [ ]:
for index, row in fairbook_ratings_.iterrows():
    if row.alt_title!="":
        ISBN = row.ISBN
        ISBN_alt = large_ISBN_dict[ISBN]
        fairbook_ratings.at[index, "ISBN"] = ISBN_alt
fairbook_ratings_correct = fairbook_ratings.drop_duplicates(["ISBN", "User-ID"])

E. Save files.

In [ ]:
 with open(location_to_save+'ISBN_dict.pkl', 'wb') as handle:
            pkl.dump(large_ISBN_dict, handle)

In [ ]:
entire_books.to_csv(location_to_save+"items_books_with_readable_properties_correct_ISBN.csv")
fairbook_books.to_csv(location_to_save+"fairbook_books_with_readable_properties_correct_ISBN.csv")

fairbook_ratings_correct.to_csv(location_to_save + 'fairbook_ratings_with_readable_properties_filtered_correct_ISBNs.csv')
entire_ratings_correct.to_csv(large_location_to_save + 'entire_ratings_with_readable_properties_filtered_correct_ISBNs.csv')